# Hotel Review Analysis

This notebook contains the code used to analyze hotel reivews. The goal is to obtain a *directed sentiment score* towards specific hotel *aspects*, such as the location, room quality, or hospitality.

## My Approach
1. Obtain reviews for hotels
2. Use the Stanford CoreNLP library to obtain sentiment scores for each sentence in a review
3. Treating a sentence as a binary tree, obtain sentiment scores for each phrase (subtree) in a sentence
4. Classify the *aspect* of each phrase
5. Aggregate results for all *aspects*, for all hotels

This approach lets me obtain directed sentiment scores for each hotel. Using this method, we can also obtain sentiment scores for several aspects, even if they appear in the same sentence or have opposite sentiment.

For example, in the sentence:
```
The food was great, but the staff was very rude.
```
The phrase "the food was great" has a positive sentiment towards the **food** aspect, while "staff was very rude" has a negative sentiment towards the **staff** aspect.

According to CoreNLP, the overall sentiment of the entire sentence is negative. However, examining subphrases allows us to accurately measure sentiment towards each aspect individually. 


## Future Work (TODO)

1. Investigate weighting by probability
2. Investigate filtering by probability
3. Investigate using neutral sentiment reviews (currently discarded in `Sentiment.java`)
4. Increase number of aspects
5. Increase regex rule coverage for aspects
6. Replace regex classifier with a neural net trained against the regex rules.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

%load_ext autoreload
%autoreload 2

%matplotlib inline

## The Dataset


In [2]:
reviews = []
urls = []
with open('data/reviews7.jl', 'r') as f:
    for line in f:
        data = json.loads(line)
        reviews.append(data['text'])
        urls.append(data['url'])

Some reviews are blank (only a title and a star rating), and others are excessively long. To combat this, I filter blank reviews and those that consist of more than 1500 characters. Experiments with all the datasets showed that 1500 characters is roughly the 95% percentile for review length. 

In [3]:
df = pd.DataFrame()
df['text'] = reviews
df['urls'] = urls
print(len(df))
print(df.text.str.len().quantile(0.95))
df = df[(df.text.str.len() > 0) & (df.text.str.len() < 1500)]
print(len(df))
df.to_csv('reviews_dev.txt', index=False)
urls = list(df.urls.values)

18068
1455.0
14094


To easily hold the data, I define a simple data wrapper class for each phrase. 

In [4]:
class Phrase:
    def __init__(self, sentiment, tokens, probability, phrase):
        self.sentiment = sentiment
        self.tokens = tokens
        self.probability = probability
        self.phrase = phrase
        self.cls = None
        self.url = None
        
    def has_class(self):
        return self.cls != None and self.cls != -1
    
    def __repr__(self):
        return 'Class={} Sentiment={} Probability={} Phrase="{}"'.format(self.cls,
                                                                       self.sentiment,
                                                                       self.probability,
                                                                       self.phrase)

## Classifying

To classify the aspect of a phrase, I used simple regex rules. These will be replaced by a neural network trained against a more comprehensive set of rules, but regex rules are enough to obtain results.

Note that the "classifier" below will only report one class. If multiple classes are detected, it will return -1. This is because we want to isolate sentiment scores towards a single aspect.

In [5]:
import re
CLASS_NAMES = ['location', 'staff', 'food', 'room']
CLASSES = np.array([i for i, _ in enumerate(CLASS_NAMES)])
def get_class(phrase_obj):
    text = phrase_obj.phrase
    
    #location patterns
    location = any([re.search(r'\blocation\b', text)])
    
    #staff patterns
    staff = any([re.search(r'\bstaff\b', text)])
    
    #food patterns
    food = any([re.search(r'\bfood\b', text)])
    
    #room quality, hospitality
    room = any([re.search(r'\bclean', text)])
    
    matches = np.array([location, staff, food, room])
    if sum(matches) == 1:
        return CLASSES[matches][0]
    else:
        return -1

When processing phrases, I want to prefer the longest phrase involving a single aspect. My other experiments have shown that the sentiment scores for longer phrases are more accurate (and have higher probability) than those for shorter phrases, especially in the presence of negating words like "not". 

The function below takes a list of phrase objects ordered from longest to shortest, and inspects their classifications. It will greedily take the longest phrase for each aspect, and ingore the rest.

In [6]:
def greedy_class_filter(ordered_phrases):
    found = [False for cls in CLASS_NAMES]
    filtered = []
    for phrase in ordered_phrases:
        if found[phrase.cls]:
            continue
        found[phrase.cls] = True
        filtered.append(phrase)

    return filtered

In order to obtain the sentiment scores for each phrase, see the ReviewSentiment Java source. There, I use CoreNLP to obtain sentiment scores for each sentence and phrase in the sentence. These phrases are obtained by running `Sentiment.java` on a cleaned version of the dataframe created above. The phrases are written out to the `reviews.tagged.txt` file, with document and sentence boundaries. 

The cell below iterates over these phrases, runs the classifier, and records the scores for each aspect.

In [7]:
from collections import defaultdict

#map from url to list of sentiment scores
hotels = defaultdict(list)

with open('reviews-nlp/ReviewSentiment/datasets/reviews.tagged.txt', 'r') as f:
    docid = 0
    buffer = []
    for line in f:
        if line.startswith('===<d'):
            docid += 1
            if docid % 1000 == 0:
                print(docid, len(hotels))
        elif line.startswith('=<s'):
            #handle each sentence
            if len(buffer) != 0:
                #construct phrase wrappers
                phrases = []
                for tokens, sentiment, prob, phrase in (p.split('|') for p in buffer):
                    phrases.append(Phrase(int(sentiment), int(tokens), float(prob), phrase))
                
                #sort by reverse length
                phrases = sorted(phrases, key = lambda x : x.tokens, reverse=True)
                
                #tag using the simple classifier
                for phrase in phrases:
                    phrase.cls = get_class(phrase)
                
                #filter phrases with no class 
                phrases = [p for p in phrases if p.has_class()]
                
                if len(phrases) > 0:
                    #record the longest phrase for each aspect in this sentence
                    filtered_phrases = greedy_class_filter(phrases)
                    hotels[urls[docid]].extend(filtered_phrases)
                    
            buffer = []
        else:
            buffer.append(line)

1000 21
2000 22
3000 25
4000 27
5000 29
6000 32
7000 34
8000 34
9000 34
10000 34
11000 34
12000 34
13000 34
14000 34


Scoring is the average sentiment towards each aspect for each hotel. Sentiment scores range from -2 to 2, with -2 being the most negative. There is a heavy bias towards good sentiment reviews, so scores for each hotel must be considered relative to other hotels.

In [8]:
hotel_scores = {}
for hotel in hotels:
    scores = [0 for _ in CLASS_NAMES]
    counts = [0 for _ in CLASS_NAMES]
    for phrase in hotels[hotel]:
        scores[phrase.cls] += phrase.sentiment
        counts[phrase.cls] += 1
    raw_scores = [s / c if c != 0 else 0 for s, c in zip(scores, counts)]
    hotel_scores[hotel] = {name : score for name, score in zip(CLASS_NAMES, raw_scores)}

## Results

Finally, the scores! The scores are mostly interpretable. Note that `Comfort_Inn_LaGuardia` has extremely low scores in each category. 

In [9]:
for hotel in hotel_scores:
    print(hotel)
    for t in hotel_scores[hotel]:
        print('Category: {:10} {:.3f}'.format(t, hotel_scores[hotel][t]))

/Hotel_Review-g48682-d224599-Reviews-Hilton_Garden_Inn_New_York_Staten_Island-Staten_Island_New_York.html
Category: location   0.545
Category: staff      0.281
Category: food       0.744
Category: room       0.537
/Hotel_Review-g60827-d2151848-Reviews-McCarren_Hotel_Pool-Brooklyn_New_York.html
Category: location   0.810
Category: staff      0.387
Category: food       0.261
Category: room       0.608
/Hotel_Review-g47369-d4447795-Reviews-Rodeway_Inn_Bronx_Zoo-Bronx_New_York.html
Category: location   0.143
Category: staff      0.500
Category: food       0.000
Category: room       0.355
/Hotel_Review-g47369-d12441510-Reviews-Comfort_Inn_Suites_near_Stadium-Bronx_New_York.html
Category: location   0.333
Category: staff      0.222
Category: food       1.000
Category: room       0.889
/Hotel_Review-g47369-d99257-Reviews-Ramada_Bronx-Bronx_New_York.html
Category: location   0.500
Category: staff      0.378
Category: food       -0.250
Category: room       0.507
/Hotel_Review-g60827-d7772856-Re